In [1]:
import zarr
import xarray as xr
import gcsfs

/opt/conda/envs/era5/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [2]:
import pandas as pd

In [3]:
# no dask. just swallow the whole thing.
#import dask
#from dask.distributed import Client
#client = Client(n_workers=13, threads_per_worker=1, memory_limit='4GB', dashboard_address=':8787')
import dask.array

In [4]:
import netCDF4 as nc
import numpy as np

In [5]:
import multiprocessing as mp

### make a new archive

In [27]:
ds_local = xr.open_dataset('2010_01_tp.nc',chunks={'time':744,'latitude':100,'longitude':100})

/opt/conda/envs/era5/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [28]:
for kk,vv in ds_local.coords.variables.items():
    print (kk,vv)

longitude <xarray.IndexVariable 'longitude' (longitude: 3600)>
array([0.000e+00, 1.000e-01, 2.000e-01, ..., 3.597e+02, 3.598e+02, 3.599e+02],
      dtype=float32)
Attributes:
    units:      degrees_east
    long_name:  longitude
latitude <xarray.IndexVariable 'latitude' (latitude: 1801)>
array([ 90. ,  89.9,  89.8, ..., -89.8, -89.9, -90. ], dtype=float32)
Attributes:
    units:      degrees_north
    long_name:  latitude
time <xarray.IndexVariable 'time' (time: 744)>
array(['2010-01-01T00:00:00.000000000', '2010-01-01T01:00:00.000000000',
       '2010-01-01T02:00:00.000000000', ..., '2010-01-31T21:00:00.000000000',
       '2010-01-31T22:00:00.000000000', '2010-01-31T23:00:00.000000000'],
      dtype='datetime64[ns]')
Attributes:
    long_name:  time


In [55]:
np.arange(90,-90,-0.1)

array([ 90. ,  89.9,  89.8, ..., -89.7, -89.8, -89.9])

In [56]:
pd.date_range('1981-01-01 00:00:00', end='2024-12-31 23:00:00',freq='1H').shape

(385704,)

In [51]:
((pd.date_range('2010-01-01', end='2024-01-01',freq='1H') - pd.Timestamp(year=2010,month=1,day=1)).total_seconds()/3600).astype(int)

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            122703, 122704, 122705, 122706, 122707, 122708, 122709, 122710,
            122711, 122712],
           dtype='int64', length=122713)

In [29]:
# get coordinate dimensions from one sample
coords = {kk:vv.values for kk,vv in ds_local.coords.variables.items()}
# overwrite the time dimension for one year
coords['time'] = pd.date_range('2010-01-01', periods=8760,freq='1H').values
coords['latitude'] = coords['latitude'][1:]

In [30]:
coords

{'longitude': array([0.000e+00, 1.000e-01, 2.000e-01, ..., 3.597e+02, 3.598e+02,
        3.599e+02], dtype=float32),
 'latitude': array([ 89.9,  89.8,  89.7, ..., -89.8, -89.9, -90. ], dtype=float32),
 'time': array(['2010-01-01T00:00:00.000000000', '2010-01-01T01:00:00.000000000',
        '2010-01-01T02:00:00.000000000', ...,
        '2010-12-31T21:00:00.000000000', '2010-12-31T22:00:00.000000000',
        '2010-12-31T23:00:00.000000000'], dtype='datetime64[ns]')}

In [ ]:
# https://xarray.pydata.org/en/stable/user-guide/io.html?appending-to-existing-zarr-stores#appending-to-existing-zarr-stores

In [ ]:
VARIABLES = {
    't2m':'2m_temperature',
    'u10':'10m_u_component_of_wind',
    'v10':'10m_v_component_of_wind',
    'tp':'total_precipitation',
    'potev':'potential_evaporation',
}

In [ ]:
# use a dask array to get the dummy dimensions
dummies = dask.array.zeros((3600,1800,744), chunks=(100,100,744))

In [ ]:
# make a mock dataset
ds = xr.Dataset(
    {kk: (tuple(coords.keys()), dummies) for kk in VARIABLES.keys()},
    coords = coords
)

In [ ]:
store = gcsfs.GCSMap(root='oxeo-era5/lk-test-resize')

In [ ]:
# Now we write the metadata without computing any array values
ds.to_zarr(store, compute=False, consolidated=True)

### push data using mp

In [6]:
ds_buffer = xr.open_dataset('data/2010_01_tp.nc',chunks={'longitude':10, 'latitude':10, 'time':35064})

/opt/conda/envs/era5/lib/python3.8/site-packages/xarray/backends/plugins.py:61: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [13]:
ds_buffer.close()

In [7]:
slices = dask.array.core.slices_from_chunks(dask.array.empty_like(ds_buffer.to_array()).chunks) # null, time, lat, lon

In [8]:
len(slices)

65160

In [9]:
slices = [s for s in slices if s[2].stop!=1801]

In [10]:
len(slices)

64800

In [11]:
slices[-1]

(slice(0, 1, None),
 slice(0, 744, None),
 slice(1790, 1800, None),
 slice(3590, 3600, None))

In [11]:
n_workers=13

In [12]:
chunk_size = len(slices)//n_workers+1
slices_rechunked = [slices[chunk_size*ii:chunk_size*(ii+1)] for ii in range(n_workers)]

In [13]:
pool = mp.Pool(n_workers)

worker_idx:11, ii_s:0worker_idx:5, ii_s:0worker_idx:6, ii_s:0

worker_idx:10, ii_s:0

worker_idx:12, ii_s:0
worker_idx:3, ii_s:0
worker_idx:7, ii_s:0
worker_idx:8, ii_s:0
worker_idx:4, ii_s:0
worker_idx:2, ii_s:0
worker_idx:1, ii_s:0
worker_idx:0, ii_s:0
worker_idx:9, ii_s:0
worker_idx:5, ii_s:0
worker_idx:11, ii_s:0
worker_idx:6, ii_s:0
worker_idx:9, ii_s:0
worker_idx:4, ii_s:0
worker_idx:1, ii_s:0worker_idx:7, ii_s:0worker_idx:0, ii_s:0


worker_idx:8, ii_s:0worker_idx:10, ii_s:0
worker_idx:3, ii_s:0

worker_idx:12, ii_s:0
worker_idx:2, ii_s:0


In [14]:
from workers import write_worker_mp

In [ ]:

"""
def write_worker_mp(filename, variable, gcs_path, slices, time_offset):
    
    # open the dataset and the zarr
    ds_worker = nc.Dataset(filename)
    store = gcsfs.GCSMap(root=gcs_path)
    z = zarr.open(store)
    
    # write each slice
    for subslice in slices:
        time_slice = subslice[1]
        offset_slice = slice(subslice[1].start+time_offset, subslice[1].stop+time_offset)
        lat_slice = subslice[2]
        lon_slice = subslice[3]
        z[variable][lon_slice, lat_slice, offset_slice] = np.transpose(
            np.squeeze(
                ds_worker[variable][time_slice, lat_slice, lon_slice]
            ),
            [2,1,0]
        )
        
        
    return 1
"""

In [39]:
args = [
    (
    '2010_02_tp.nc', 
    'tp', 
    'oxeo-era5/lk-test-resize', 
    slices_rechunked[ii], 
    744,
    ii
    ) for ii in range(n_workers)
]
print (len(args))

13


In [40]:
import time

In [41]:
# let see how long this takes
tic = time.time()
pool.starmap(write_worker_mp, args)
toc = time.time()
print (toc-tic)

127.50109648704529


### test resize

In [59]:
store = gcsfs.GCSMap(root='oxeo-era5/lk-test-resize')

In [60]:
z = zarr.open(store)

In [42]:
zxr = xr.open_zarr(store)

KeyError: 'Zarr object is missing the attribute `_ARRAY_DIMENSIONS`, which is required for xarray to determine variable dimensions.'

In [26]:
zxr

,Array,Chunk
Bytes,35.92 GiB,56.76 MiB
Shape,"(3600, 1800, 744)","(100, 100, 744)"
Count,649 Tasks,648 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,35.92 GiB,56.76 MiB
Shape,"(3600, 1800, 744)","(100, 100, 744)"
Count,649 Tasks,648 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [29]:
z['time'].resize(744+672)

In [30]:
z['time'].shape

(1416,)

In [34]:
z['time'] = pd.date_range('2010-01-01', periods=744+672,freq='1H').values

In [36]:
shp = z['tp'].shape

In [37]:
z['tp'].resize(shp[0], shp[1], shp[2]+672)

In [38]:
z['tp'].shape

(3600, 1800, 1416)

In [ ]:
z['time'].attrs['_ARRAY_DIMENSIONS']

In [ ]:
z['time']

In [43]:
store = gcsfs.GCSMap(root='oxeo-era5/lk-test-direct')

In [44]:
z = zarr.open(store)